
# Universidad de Monterrey

## División de Ingenierías

### Lab - Robotics

#### Lab 8: image edge detection using op

Authors: Jesús Ramírez, Kassandra Ibarra, Alberto Herrera

##### Introducción
La detección de bordes es una da las herramientas de mayor utilidad en los sistemas de visión computacional, principalmente en los sistemas dinámicos o móviles. La detección de bordes permite a un robot, sistema o máquina, comprender un poco más el ambiente en el que se encuentra ya que permite reconocer los límites de un objeto.
La detección de bordes que realizaremos en esta práctica es muy similar a los ejercicios de la práctica pasada, ya que se utilizan los métodos de correlación y convolución con kernels específicos para lograr la detección de bordes.

##### Objetivo 
El objetivo de esta práctica es aprender sobre la detección de bordes en una imagen con lo que se espera poder tener la capacidad de detectar la figura de los objetos en una imagen. De esta manera se facilitará el uso de funciones tanto de filtros y operadores para poder ser implementada en el proyecto final cuando sea necesario reconocer los bordes de la figura rectangular de las cajas

##### Materiales
- Raspberry PI 3
- Pantalla y cable HDMI para visualizar la raspberry.
- Además, la Raspberry debe contar con el software pedido al inicio del laboratorio (opencv, python 3.5, Jupyter, ssh, entre otros).

##### Procedimiento 
Para poder detectar los bordes de los objetos en una imagen se deben de considerar diferentes factores como:
- discontinuidad de superficie
- discontinuidad de profundidad
- discontinuidad de color
- discontinuidad de iluminación
Para detectar estas discontinuidades es necesario analizar las regiones de una imagen en las que existan cambios abruptos entre píxeles, para esto nos podemos apoyar en las derivadas ya que estas muestran la dirección en la que una función cambia repentinamente. Es por esta razón por la que se utilizarán funciones derivativas en esta práctica.
 
##### Gradiente
Al momento de detectar bordes en una imagen predeterminada se utilizan principalmente funciones que dan como resultado la derivada de la imagen no solo porque la derivada nos permite ver los cambios abruptos en una región de la imagen si no porque al utilizar operadores derivativos también nos permite utilizarlos como máscaras o kernels en las que se pueden aplicar convoluciones o correlaciones con el fin de encontrar el gradiente de la imagen dada por la función: 
![title](grad.png)

En openCV se puede utilizar el método:

el cual te permite realizar la convolución de de una imagen con un operador derivativo del kernel de los cuales se pueden utilizar los siguientes: 
- El operador Sobel
- El operador Prewitt
- El operador Roberts 


##### Operador Sobel 
 
En esta sección de la práctica profundizamos en el uso de la función:
<h3 align="center"> cv2.Sobel() </h3>

en lugar del método cv2.filter2D() mencionado anteriormente ya que al aprender a utilizar la operación sobel se tienen las bases para implementar los métodos de Prewitt y Roberts. 
 
A continuación se muestra el resultado de utilizar el código propuesto con la función Sobel, añadiendo la definición para desplegar las imágenes con matplotlib.pyplop, este código también incluye el uso de los operadores Roberts y Prewitt, más abajo se explican sus salidas:

In [ ]:
"""
	image_edge_detection_using_operators.py
	author: andres.hernandezg@udem.edu
	universidad de monterrey
"""

# import required libraries
import numpy as np
import matplotlib.pyplot as plt
import cv2


# compute Sobel gradient
def compute_absolute_sobel_gradient(img, ax='x', ksize=3, threshold=(40,140)):

	# 1) check whether img is a colour or greyscale image
	if len(img.shape)>2:

		# convert from colour to greyscale image
		grey = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

	else:
		# if greyscale grey=img
		grey = img

	# 2) take the derivate in 'ax' axis
	if ax.lower()=='x':

		# apply the Sobel operator along the x axis
		sobel_derivative = cv2.Sobel(grey, ddepth=cv2.CV_64F, dx=1, dy=0, ksize=ksize, scale=1)

	if ax.lower()=='y':

		# apply the Sobel operator along the y axis
		sobel_derivative = cv2.Sobel(grey, ddepth=cv2.CV_64F, dx=0, dy=1, ksize=ksize, scale=1)

	# 3) take the absolute value of the derivative
	sobel_absolute = np.absolute(sobel_derivative)

	# 4) scale to 8-bit (0-255), then convert to type = np.uint8
	sobel_scaled = np.uint8(255 * sobel_absolute / np.max(sobel_absolute))

	# 5) create a mask of 1's where threshold[0] < sobel_scaled < threshold[1]
	binary_output = np.zeros_like(sobel_scaled)
	threshold_min = threshold[0]
	threshold_max = threshold[1]
	binary_output[(sobel_scaled >= threshold_min) & (sobel_scaled <= threshold_max)] = 1

    # return binary_image with gradient being detected along 'ax' axis
	return sobel_derivative, binary_output


# combine x and y derivatives using AND operation
def combine_x_and_y_binary_derivatives(img_derivative_x, img_derivative_y, threshold=(40,120)):

	# verify that both image derivatives are the same size
	if img_derivative_x.shape != img_derivative_y.shape:
		print('ERROR [combine_x_and_y_binary_derivatives]: img_binary_x and img_binary_y images should be of same size')
		exit()

	# 1) take the absolute value of the derivative
	absolute_x = np.absolute(img_derivative_x)
	absolute_y = np.absolute(img_derivative_y)

	# 2) scale to 8-bit (0-255), then convert to type = np.uint8
	absolute_scaled_x = np.uint8(255 * absolute_x / np.max(absolute_x))
	absolute_scaled_y = np.uint8(255 * absolute_y / np.max(absolute_y))

	# 3) create a mask of 1's where threshold[0] < sobel_scaled < threshold[1]
	binary_combined_gradients = np.zeros_like(absolute_scaled_x)
	indx_x = (absolute_scaled_x >= threshold[0]) & (absolute_scaled_x <= threshold[1])
	indx_y = (absolute_scaled_y >= threshold[0]) & (absolute_scaled_y <= threshold[1])
	binary_combined_gradients[indx_x|indx_y] = 1

	# return combined binary x-and-y derivatives
	return binary_combined_gradients


# compute magnitude of x and y derivatives
def compute_magnitude_of_derivatives(img_derivative_x, img_derivative_y, ksize=3, threshold=(40,120)):

	# 1) calculate the magnitude
    gradient_magnitude = np.sqrt(np.power(img_derivative_x, 2) + np.power(img_derivative_y, 2))

    # 2) scale to 8-bit (0 - 255) and convert to type = np.uint8
    gradient_scaled = np.uint8(255 * gradient_magnitude / np.max(gradient_magnitude))

    # 3) create a binary mask where mag thresholds are met
    binary_magnitude = np.zeros_like(gradient_scaled)
    binary_magnitude[(gradient_scaled >= threshold[0]) & (gradient_scaled <= threshold[1])] = 1

    return binary_magnitude


# compute orientation of magnitude of x and y derivatives
def compute_direction(img_derivative_x, img_derivative_y, threshold=(0, np.pi/2)):

	# 1) take the absolute value of the x and y gradients
    gradient_magnitude_x = np.absolute(img_derivative_x)
    gradient_magnitude_y = np.absolute(img_derivative_y)

    # 2) calculate the gradient magnitude direction
    #gradient_direction = np.arctan2(gradient_magnitude_y, gradient_magnitude_x)
    gradient_direction = np.arctan2(img_derivative_y, img_derivative_x)

    # 3) reate a binary mask where direction thresholds are met
    binary_direction = np.zeros_like(gradient_direction)
    binary_direction[(gradient_direction >= threshold[0]) & (gradient_direction <= threshold[1])] = 1

    return binary_direction
def visualise_image(img, fig_number, title, flag_colour_conversion, conversion_colour, cmap):
	
	plt.figure(fig_number)
	if flag_colour_conversion:
		img=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
	if cmap == '':
		cmap = 'gray'
	plt.imshow(img,cmap)
	plt.title(title)
	plt.xticks([])
	plt.yticks([])
	#plt.show()	
	return None
def edge_detection(image,kernel1,kernel2,fig_number, title, flag_colour_conversion, conversion_colour, cmap):
	image = cv2.filter2D(image,-1,kernel1)
	image = cv2.filter2D(image,-1,kernel2)
	visualise_image(image,fig_number, title, flag_colour_conversion, conversion_colour, cmap)
	return None
# pipeline
def run_pipeline(img_name):

	# read image
	img = cv2.imread(img_name)

	# verify that image `img` exist
	if img is None:
		print('ERROR: image ', img_name, 'could not be read')
		exit()



	# compute sobel derivative along x axis
	img_derivative_x, img_binary_x = compute_absolute_sobel_gradient(img, ax='x', ksize=3, threshold=(40,140))

	# compute sobel derivative along y axis
	img_derivative_y, img_binary_y = compute_absolute_sobel_gradient(img, ax='y', ksize=3, threshold=(40,140))

	# combine x and y derivatives
	img_combined_derivatives = combine_x_and_y_binary_derivatives(img_derivative_x, img_derivative_y, threshold=(50,140))

	# compute magnitude of gradient
	img_magnitude_gradient = compute_magnitude_of_derivatives(img_derivative_x, img_derivative_y, ksize=3, threshold=(40,140))

	# compute direction of gradient
	thresh_min = 0
	thresh_max = 2
	img_direction_gradient = compute_direction(img_derivative_x, img_derivative_y, threshold=(np.radians(thresh_min), np.radians(thresh_max)))
	#Kernel Roberts
	roberts1 = np.array([[0,1],[-1,0]])
	roberts2 = np.array([[1,0],[0,-1]])
	#Kernel	prewwit
	Gx = np.array([[-1,0,1],[-1,0,1],[-1,0,1]])
	Gy = np.array([[1,1,1],[0,0,0],[-1,-1,-1	]])
	
	visualise_image(img, 1, 'Colour input image', True, 'BGR2RGB', '')
	visualise_image(img_derivative_x, 2, 'x derivative', False, '', 'gray')
	visualise_image(img_binary_x, 3, 'binary x derivative', False, '', 'gray')
	visualise_image(img_derivative_y, 4, 'y derivative', False, '', 'gray')
	visualise_image(img_binary_y, 5, 'binary y derivative', False, '', 'gray')
	visualise_image(img_combined_derivatives, 6, 'combined x and y derivative', False, '', 'gray')
	visualise_image(img_magnitude_gradient,7,'gradient magnitude',False,'','gray')
	edge_detection(img,roberts1,roberts2,8,'Roberts Kernel',False,'','gray')
	edge_detection(img,Gx,Gy,9,'Prewitt Kernel',False,'','gray')
	plt.show()

# uncomment the corresponding line to try a particular image
#img_name = 'opera_house_vivid_sydney.jpg'
#img_name = 'sydney_harbour.jpg'
img_name = 'vehicular_traffic.jpg'

# run pipeline
run_pipeline(img_name)

![title](todos_bordes.png)

Al aplicar las derivadas en x y en y podemos ver que se obtienen los relieves de los objetos de la imagen respecto al eje en el que se aplica la derivada, esto lo podemos notar de mejor maera en la derivada binaria, ya que en estas permanecen las lineas perpendiculares al eje en el que se aplicó la derivada, es decir, en la derivada binaria de x quedan solo lineas verticales mientras que en la de y solo horizontales.
En la derivada combinada pareciera la suma del resultado de las dos derivadas, se obtienen los bordes tanto horizontales como verticales, lo que sucede al combinarlas es que permanecen los pixeles en blanco que no se sobreponen entre sí.
Por otro lado, en la dirección del gradiente y magnitud, creemos que son solo información relevante acerca de la detección de bordes, pero no crucial o útil para la detección tal cual, ya que la dirección del gradiente arroja solo puntos, por otro lado la magnitud del gradiente es prácticamente la misma que las derivadas combinadas de x y y.

En esta imagen se muestra el resultado de utilizar los operadores Prewitt y Roberts con la función filter2D y en las secciones correspondientes se explica la salida de cada uno:
![title](Roberts_Prewitt.png)


#### Roberts
Al utilizar el operador de Roberts pareciera que no hace nada o que no detecta bordes, talvez lo que permie el operador Roberts es detectar bordes más complicados dejando atras los más fáciles de ver.
#### Prewitt
Por otro lado el Prewitt si bien no marco los bordes más delgados o más difíciles, como los carros o el bosque, los bordes que detectó los entregó muy claros o con bastante contraste respecto al fondo, creo que el rendimiento del filtro de Prewitt es aceptable si estas dispuesto a renunciar a una detección más profunda a cambio de bordes mejor definidos.

#### Conclusiones
##### Kassandra Dzuara Ibarra Ortiz 
Esta práctica en general me pareció muy interesante porque explica cómo es que funcionan muchas aplicaciones que existen en el mercado sobretodo en el área automotriz en los carros con vidrios inteligentes que te detectan ciclistas y otros vehículos con el fin de prevenir accidentes, También el hecho de que te explica cómo es que se utiliza el cálculo diferencial es algo interesante ya que te permite aterrizar conocimientos anteriores. Además, en el caso de esta práctica si sera de las que tendrá más aplicación en nuestro proyecto final ya que le permitirá a nuestro robot móvil el poder detectar los bordes de las cajas de colores que tiene que recoger por el camino.

##### Jesus Alejandro Ramirez Castañeda


##### Alberto Jasiel Herrera Michel
Esta práctica fue lo que yo esperaba aprender cuando nos dijeron sobre lo que iba a tratar el curso, creo que es  una de las prácticas en las que adquirimos un conocimiento que muy seguramente se aplique en casi cualquier aplicación de la visión computacional. Del mismo modo, dada su complejidad también fue una de las que más nos costó entender, pero es muy interesante como en este tema (y en casi todos los de visión) se aplica cálculo diferencial y en general cálculo para procesar la imagen.

En este semestre estamos haciendo un proyecto de un robot móvil que tiene que desplazarse, creo que la detección de bordes es crucial para el correcto funcionamiento de este, al detectar los bordes del "laberinto" en el que lo tendremos podrá determinar cual es el camino.

Por último, vemos como es muy poca la diferencia que existe entre el filtrado y la detección de bordes, a mi parecer la detección de bordes es un filtrado especializado.